In [1]:
import pandas as pd
import numpy as np
from pandas_datareader import data, wb
from datetime import datetime
import scipy.stats as stats

In [36]:
#pull in dataset used as the benchmark and dataset for SMA strategy. Note the start and end dates.

start = datetime(2005,1,1)
end = datetime(2020,5,1)
stock = data.DataReader("IVV", 'yahoo', start, end).reset_index()
close = stock["Close"] #data used for SMA function

In [37]:
#The SMA function,includes current day included SMA. basic form is to compare price to SMA of length set by user.
#This function looks for another price signal after a number of days equal to the return length

def SMA_timing_func(SMA_len, return_len, data, initial):
    timing = initial
    hold = 'cash'
    #acct_value = []
    
    for n in range (SMA_len, len(data)-return_len-np.mod(len(data)-SMA_len, return_len),return_len): #the stopping point maximises the loops us of dtaset without falling outside of it.
        price = data.iloc[n]
        next_price = data.iloc[n+return_len]
        SMA = data[n-SMA_len:n].mean()
        #acct_value.append(timing)
        if (price >= SMA) & (hold =='stock'): #the first case for when the loop checks for a price signal
            timing = next_price/price*timing
        elif (price >= SMA) & (hold =='cash'):#the second case for when the loop checks for a price signal
            hold = 'stock'
            timing = next_price/price*timing
        elif (price < SMA) & (hold =='stock'):#the third and final case for when the loop checks for a price signal because in th fourth scenario nothing happens.
            hold ='cash'
    return(timing)

In [38]:
#Here you can set how finely you want to tune the parameters
SMA_range = np.arange(1,731, 73) #1,10,73
return_range = np.arange(1,91, 30) #1,10,30 

In [39]:
#create dataframe that has sma and return ranges as coloumns. output to a third vector that gets concatenated to dataframe

In [74]:
#simple version, varying over parameter space
def vary_over_space(data):
    Max_Val = 0
    num = 0
    outperform=0
    benchmark = data.iloc[-1]/data.iloc[0]
    for i in return_range:
        for j in SMA_range:
            Value=SMA_timing_func(j,i,data,1)
            if Value > benchmark:
                print(Value)
                print(Value, "outperforms at SMA =",j,", and return_range =", i)
            if Value > Max_Val:
                Max_Val = Value
                optimum_SMA=j
                optimum_return=i
        num=num+1
        print("Outer loop number", num)
    print(Max_Val)
    print(optimum_SMA) 
    print(optimum_return)
    print("Benchmark is ", benchmark)

In [75]:
vary_over_space(close)

Outer loop number 1
2.6759075145410565
2.6759075145410565 outperforms at SMA = 147 , and return_range = 31
2.897752724327385
2.897752724327385 outperforms at SMA = 220 , and return_range = 31
2.6656978739658177
2.6656978739658177 outperforms at SMA = 366 , and return_range = 31
2.4462965986598495
2.4462965986598495 outperforms at SMA = 512 , and return_range = 31
Outer loop number 2
2.4180859207828465
2.4180859207828465 outperforms at SMA = 366 , and return_range = 61
2.6134546043843305
2.6134546043843305 outperforms at SMA = 439 , and return_range = 61
Outer loop number 3
2.897752724327385
220
31
Benchmark is  2.353727386138126


In [ ]:
#This methods is extremely vulverable to overfitting. Need to look for a more robust system
#Might be interesting to make a 3d graph of the reurn over parameter space